In [93]:
import pandas as pd
import calmap
import numpy as np
import os

In [94]:
from pathlib import Path
from os import environ, write

import re
from datetime import datetime, time
import datetime as dt
from collections import defaultdict

import requests

In [114]:
def load_data():
    if environ.get("USE_LOCAL_FILES"):
        paths = Path("extracted").rglob("*.csv")
        df_parts = []

        for path in sorted(paths):
            print("Reading for analysis:", path)

            with open(path, "r", encoding="utf-8") as fp:
                df_part = pd.read_csv(fp, delimiter=",", dtype=str)
                df_parts.append(df_part)

            # if df is None:
            #     df = df_part
            # else:
            #     df = df.append(df_part, ignore_index=True)

        df = pd.concat(df_parts, ignore_index=True)
        
    

    else:
        raise NotImplementedError("Remote data not yet implemented")
        
    df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors = 'coerce')
    df['date_of_proceeding'] = pd.to_datetime(df['date_of_proceeding'], errors = 'coerce')
    df['date_of_publication'] = pd.to_datetime(df['date_of_publication'], errors = 'coerce')
    
    #   df['zipcode'].astype(str)

    return df

In [115]:
def run():
    df = load_data()

#Alternative 
# df[['order_date','receive_date']] = df[['order_date','receive_date']].apply(pd.to_datetime)


    # Print dataframe sample and info
    pd.set_option("display.max_columns", 100)
    pd.set_option("display.width", 1000)
    pd.set_option("display.max_rows", 200)

    print(df)
    print(df.info())
    print(df.describe())
    
    print(df.sort_values(by="date_of_proceeding", ascending=False).head())
    
    ik_privat(df)
    in_unternehmen(df)

In [116]:
df = load_data()

Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-01-17T08-05-41.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-01-24T08-05-43.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-01-31T03-50-40.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-02-07T03-47-56.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-02-14T03-54-29.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-02-21T04-08-08.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-02-28T04-20-16.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-03-07T07-20-48.jsonl.csv
Reading for analysis: extracted/buckets/insolven

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100151 entries, 0 to 1100150
Data columns (total 17 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   _key                 83085 non-null    object        
 1   case_nr              1100151 non-null  object        
 2   court                1100151 non-null  object        
 3   date_of_birth        810973 non-null   datetime64[ns]
 4   date_of_proceeding   977144 non-null   datetime64[ns]
 5   date_of_publication  1100151 non-null  datetime64[ns]
 6   federal_state        1100151 non-null  object        
 7   file_name            983465 non-null   object        
 8   kind                 1099603 non-null  object        
 9   title                983465 non-null   object        
 10  type_of_proceeding   1100151 non-null  object        
 11  zipcode              1095061 non-null  object        
 12  detail_form_name     116686 non-null   object        
 1

In [118]:
# Funktion in_kind mit dem Argument/Wert df wird definiert
def in_kind(df):
    in_unternehmen = (df[df['kind'] == 'in'])
    return in_unternehmen

# Funktion def wird ausgeführt
data_in = in_kind(df)

In [7]:
# convert series to df
data_in_2 = pd.DataFrame(data_in)

In [119]:
new_data = data_in.replace(['Nordrhein-Westfalen'], 'nw')

In [120]:
new_data.federal_state.unique()

array(['ns', 'he', 'nw', 'mv', 'th', 'bb', 'be', 'rp', 'by', 'sn', 'sh',
       'st', 'hh', 'hb', 'sl', 'bw', 'Bayern', 'Rheinland-Pfalz',
       'Thüringen', 'Schleswig-Holstein', 'Sachsen-Anhalt', 'Sachsen',
       'Saarland', 'Niedersachsen', 'Mecklenburg-Vorpommern', 'Hessen',
       'Hamburg', 'Bremen', 'Brandenburg', 'Berlin', 'Baden-Württemberg'],
      dtype=object)

In [121]:
data_nw_in = data_in[(data_in.federal_state == 'nw')]

In [11]:
data_nw_in.tail()

,_key,case_nr,court,date_of_birth,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
983425,NaN,0093_IN00188_17,agaachen,NaT,2021-07-21,2021-07-26,nw,gerichte/nw/agaachen/17/0093_IN00188_17/2021_0...,in,"Wolf, Michael, Aachen, 93 IN 188/17",Restschuldbefreiung,52159,NaN,NaN,NaN,NaN,NaN
983428,NaN,0080_IN00322_14,agbochum,NaT,2021-07-23,2021-07-26,nw,gerichte/nw/agbochum/14/0080_IN00322_14/2021_0...,in,"Wurzer Speditions GmbH, Bochum, 80 IN 322/14, ...",Bestimmung_Termine,44894,NaN,NaN,NaN,NaN,NaN
983447,NaN,0167_IN00030_16,agessen,1972-12-29,2021-07-23,2021-07-26,nw,gerichte/nw/agessen/16/0167_IN00030_16/2021_07...,in,"Zimmermann, Tim, Essen, 167 IN 30/16",Restschuldbefreiung,45131,NaN,NaN,NaN,NaN,NaN
983448,NaN,0167_IN00030_16,agessen,1972-12-29,2021-07-23,2021-07-26,nw,gerichte/nw/agessen/16/0167_IN00030_16/2021_07...,in,"Zimmermann, Tim, Essen, 167 IN 30/16",Entscheidungen_im_Verfahren,45131,NaN,NaN,NaN,NaN,NaN
983458,NaN,0099_IN00040_15,agbonn,NaT,2021-07-22,2021-07-26,nw,gerichte/nw/agbonn/15/0099_IN00040_15/2021_07_...,in,"Özata, Adem, Troisdorf, 99 IN 40/15",Restschuldbefreiung,53842,NaN,NaN,NaN,NaN,NaN


In [12]:
data_nw_in.sample(5)

,_key,case_nr,court,date_of_birth,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
780126,NaN,0071_IN00384_15,agkoeln,NaT,2021-05-07,2021-05-11,nw,gerichte/nw/agkoeln/15/0071_IN00384_15/2021_05...,in,"Angermund, Ina Maria, Leverkusen, 71 IN 384/15",Restschuldbefreiung_Erteilung_Versagung,51371,NaN,NaN,NaN,NaN,NaN
9434,491759/1/10/9434,0061_IN00167_18,agduisb,1993-02-18,2021-01-12,2021-01-13,nw,gerichte/nw/agduisb/18/0061_IN00167_18/2021_01...,in,"Kilic, Tolga, Mülheim an der Ruhr, 61 IN 167/18",Restschuldbefreiung,45481,NaN,NaN,NaN,NaN,NaN
613238,NaN,0165_IN00016_15,agessen,NaT,2021-04-16,2021-04-19,nw,gerichte/nw/agessen/15/0165_IN00016_15/2021_04...,in,"Werner, Patrick Manfred, Essen, 165 IN 16/15",Restschuldbefreiung,45141,NaN,NaN,NaN,NaN,NaN
315392,NaN,0097_IN00106_18,agbonn,NaT,2021-03-01,2021-03-02,nw,gerichte/nw/agbonn/18/0097_IN00106_18/2021_03_...,in,"Die Energieagenten Versorgungs-GmbH, Troisdorf...",Bestimmung_Termine,12961,NaN,NaN,NaN,NaN,NaN
718287,NaN,0145_IN00191_21,agwupper,1972-05-12,2021-05-10,2021-05-12,nw,gerichte/nw/agwupper/21/0145_IN00191_21/2021_0...,in,"Bolosis, Vasileios, Wülfrath, 145 IN 191/21",Eroeffnung_Insolvenzverfahren,42489,NaN,NaN,NaN,NaN,NaN


## Anzahl der Vorgänge bei Unternehmensinsolvenzen von Januar bis Mitte September

In [122]:
#data_nw_in.type_of_proceeding.unique()
data_nw_in['type_of_proceeding'].value_counts()

Restschuldbefreiung                                  15786
Entscheidungen_im_Verfahren                           8865
Bestimmung_Termine                                    8716
Entscheidungen_im_Verfahren_Aufhebung_Einstellung     7152
Entscheidungen_im_Verfahren_mit_Termine               6516
Verwalter_Treuhaender_Verteilungsverzeichnis          5963
Eroeffnung_Insolvenzverfahren                         5461
Sonstiges                                             1686
Abweisungen_mangels_Masse                             1588
Anordnung_Sicherungsmassnahmen                        1212
Restschuldbefreiung_Erteilung_Versagung                760
Verteilungsverzeichnis_Nachtragsvert_Loeschung         461
Sicherungsmassnahme_Aufhebung                          365
Sonstiges_ausserhalb_des_Verfahresn                    355
Entscheidungen_nach_Aufhebung_des_Verfahrens            96
InsO_Plan_Aufhebung_Ueberwachung                        19
InsO_Plan_Ankuendigung_Ueberwachung                     

### Unternehmensinsolvenz Eröffnungen von 01.01. - 10.09. in NRW

In [123]:
def in_eröffnung(data_nw_in):
    in_eröffnet = (data_nw_in[data_nw_in['type_of_proceeding'] == 'Eroeffnung_Insolvenzverfahren'])
    return in_eröffnet

# Funktion def wird ausgeführt
data_nw_eröffnet = in_eröffnung(data_nw_in)

In [154]:
data_nw_eröffnet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5461 entries, 123 to 970081
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   _key                 341 non-null    object        
 1   case_nr              5461 non-null   object        
 2   court                5461 non-null   object        
 3   date_of_birth        3985 non-null   datetime64[ns]
 4   date_of_proceeding   5455 non-null   datetime64[ns]
 5   date_of_publication  5461 non-null   datetime64[ns]
 6   federal_state        5461 non-null   object        
 7   file_name            5461 non-null   object        
 8   kind                 5461 non-null   object        
 9   title                5461 non-null   object        
 10  type_of_proceeding   5461 non-null   object        
 11  zipcode              5461 non-null   object        
 12  detail_form_name     0 non-null      object        
 13  format               0 non-nu

### date_of_publication

In [125]:
freq_in_nw_eröffnung = data_nw_eröffnet.groupby(['date_of_publication'], as_index=False).count() 
freq_in_nw_eröffnung.sample(5)

,date_of_publication,_key,case_nr,court,date_of_birth,date_of_proceeding,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
43,2021-03-03,0,98,98,66,98,98,98,98,98,98,98,0,0,0,0,0
7,2021-01-13,36,36,36,28,36,36,36,36,36,36,36,0,0,0,0,0
63,2021-03-30,0,32,32,26,32,32,32,32,32,32,32,0,0,0,0,0
115,2021-06-17,0,50,50,36,50,50,50,50,50,50,50,0,0,0,0,0
53,2021-03-17,0,52,52,42,52,52,52,52,52,52,52,0,0,0,0,0


In [127]:
freq_in_nw_eröffnung.to_csv('freq_in_nw_eröffnung.csv')

#### Group_by date_of_birth Eröffnung

In [156]:
freq_in_nw_eröffnung_birth.isNaN().sum()

AttributeError: 'DataFrame' object has no attribute 'isNaN'

In [138]:
# convert str to date
freq_in_nw_eröffnung_birth = data_nw_eröffnet.copy()
freq_in_nw_eröffnung_birth.date_of_birth = freq_in_nw_eröffnung_birth.date_of_birth.dt.year

In [139]:
freq_in_nw_eröffnung_birth = freq_in_nw_eröffnung_birth.groupby(['date_of_birth'], as_index=False).count()
freq_in_nw_eröffnung_birth.sample(10)

,date_of_birth,_key,case_nr,court,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
30,1956.0,1,45,45,45,45,45,45,45,45,45,45,0,0,0,0,0
73,1999.0,1,10,10,10,10,10,10,10,10,10,10,0,0,0,0,0
28,1954.0,0,30,30,30,30,30,30,30,30,30,30,0,0,0,0,0
10,1935.0,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
67,1993.0,1,55,55,55,55,55,55,55,55,55,55,0,0,0,0,0
66,1992.0,7,62,62,62,62,62,62,62,62,62,62,0,0,0,0,0
75,2011.0,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
41,1967.0,9,104,104,104,104,104,104,104,104,104,104,0,0,0,0,0
55,1981.0,5,104,104,104,104,104,104,104,104,104,104,0,0,0,0,0
1,1920.0,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0


In [142]:
#freq_in_nw_eröffnung_birth['date_of_birth'] = freq_in_nw_eröffnung_birth['date_of_birth'].replace('\.0', '', regex=True)
#freq_in_nw_eröffnung_birth.sample(5)

,date_of_birth,_key,case_nr,court,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
45,1971.0,3,108,108,108,108,108,108,108,108,108,108,0,0,0,0,0
32,1958.0,1,48,48,48,48,48,48,48,48,48,48,0,0,0,0,0
21,1947.0,0,4,4,4,4,4,4,4,4,4,4,0,0,0,0,0
38,1964.0,7,71,71,71,71,71,71,71,71,71,71,0,0,0,0,0
75,2011.0,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0


In [ ]:
# Berechnen wie viele NaN 
# Dublikate in date_of_birth u.date_of_publication entfernen --> extractor

In [46]:
freq_in_nw_eröffnung_birth.to_csv('freq_in_nw_eröffnung_birth.csv')

#### Eröffnung group_by zipcode 

In [143]:
freq_in_nw_eröffnung_zipcode = data_nw_eröffnet.groupby(['zipcode'], as_index=False).count()
freq_in_nw_eröffnung_zipcode.sample(10)

,zipcode,_key,case_nr,court,date_of_birth,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,detail_form_name,format,name,register,request_fingerprint
462,40668,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
372,32756,0,6,6,4,6,6,6,6,6,6,6,0,0,0,0,0
857,50858,1,4,4,4,4,4,4,4,4,4,4,0,0,0,0,0
176,17812,0,2,2,0,2,2,2,2,2,2,2,0,0,0,0,0
387,33142,1,9,9,9,9,9,9,9,9,9,9,0,0,0,0,0
863,50939,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
980,53840,0,12,12,10,12,12,12,12,12,12,12,0,0,0,0,0
1121,73106,0,2,2,0,2,2,2,2,2,2,2,0,0,0,0,0
65,12680,0,2,2,0,2,2,2,2,2,2,2,0,0,0,0,0
306,29790,0,2,2,0,2,2,2,2,2,2,2,0,0,0,0,0


In [144]:
freq_in_nw_eröffnung_zipcode.to_csv('freq_in_nw_eröffnung_zipcode.csv')

#### Restschulderlassung von 01.01. - 10.09. in NRW

In [47]:
def in_restschuld(data_nw_in):
    in_restschuld_frei = (data_nw_in[data_nw_in['type_of_proceeding'] == 'Restschuldbefreiung'])
    return in_restschuld_frei

# Funktion def wird ausgeführt
restschuld_in = in_restschuld(data_nw_in)

In [48]:
restschuld_in.describe()

,_key,case_nr,court,date_of_birth,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
count,1327,15786,15786,4285,15757,15786,15786,15786,15786,15786,15786,15786,0,0,0,0,0
unique,1327,5488,19,1870,166,135,1,8401,1,5549,1,989,0,0,0,0,0
top,491759/1/10/37449,0073_IN00013_15,agkoeln,1964-05-25 00:00:00,2021-04-20 00:00:00,2021-06-10 00:00:00,nw,gerichte/nw/agdortmd/20/0255_IN00049_20/2021_0...,in,"Kaplan, Baris, Waltrop, 145 IN 673/15",Restschuldbefreiung,50374,NaN,NaN,NaN,NaN,NaN
freq,1,12,2407,12,196,212,15786,2,15786,10,15786,100,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,1932-04-26 00:00:00,2020-12-21 00:00:00,2021-01-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,1999-06-26 00:00:00,2021-08-06 00:00:00,2021-08-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
freq_in_restschuld = restschuld_in.groupby(['date_of_publication'], as_index=False).count() 
print(freq_in_restschuld)

    date_of_publication  _key  case_nr  court  date_of_birth  \
0            2021-01-04    51       51     51             15   
1            2021-01-05    92       92     92             25   
2            2021-01-06   105      105    105             41   
3            2021-01-07    70       70     70             18   
4            2021-01-08    57       57     57             17   
..                  ...   ...      ...    ...            ...   
130          2021-08-03     0       40     40             11   
131          2021-08-04     0       47     47             13   
132          2021-08-05     0       40     40              7   
133          2021-08-06     0       29     29              6   
134          2021-08-09     0        2      2              1   

     date_of_proceeding  federal_state  file_name  kind  title  \
0                    51             51         51    51     51   
1                    92             92         92    92     92   
2                   105          

In [145]:
freq_in_nw_restschuld_zipcode = restschuld_in.groupby(['zipcode'], as_index=False).count()
freq_in_nw_restschuld_zipcode.sample(10)

,zipcode,_key,case_nr,court,date_of_birth,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,detail_form_name,format,name,register,request_fingerprint
734,52353,5,46,46,22,46,46,46,46,46,46,46,0,0,0,0,0
126,33378,2,20,20,2,20,20,20,20,20,20,20,0,0,0,0,0
949,65239,0,2,2,0,2,2,2,2,2,2,2,0,0,0,0,0
540,47798,5,38,38,12,38,38,38,38,38,38,38,0,0,0,0,0
661,50933,0,8,8,0,8,8,8,8,8,8,8,0,0,0,0,0
501,47119,0,10,10,4,10,10,10,10,10,10,10,0,0,0,0,0
924,59505,0,4,4,0,4,4,4,4,4,4,4,0,0,0,0,0
197,40489,0,2,2,0,2,2,2,2,2,2,2,0,0,0,0,0
81,32427,2,6,6,2,6,6,6,6,6,6,6,0,0,0,0,0
196,40479,0,12,12,4,12,12,12,12,12,12,12,0,0,0,0,0


In [50]:
freq_in_restschuld.to_csv('freq_in_restschuld.csv')

In [147]:
freq_in_nw_restschuld_zipcode.to_csv('freq_in_restschuld_zipcode.csv')

####  Vorher Geburtstage Fixen !! Restschuld by birth

In [52]:
freq_in_restschuld_birth = restschuld_in.groupby(['date_of_birth'], as_index=False).count() 
freq_in_restschuld_birth.sample(5)

,date_of_birth,_key,case_nr,court,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
394,1962-04-13,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0
323,1960-09-22,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
504,1964-03-29,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
1161,1975-11-09,0,4,4,4,4,4,4,4,4,4,4,0,0,0,0,0
1527,1982-09-17,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0


In [53]:
freq_in_restschuld_birth.to_csv('freq_in_restschuld_birth.csv')

#### Abweisung Mangelsmasse in NRW

In [148]:
def in_abweisung(data_nw_in):
    in_abweisung_masse = (data_nw_in[data_nw_in['type_of_proceeding'] == 'Abweisungen_mangels_Masse'])
    return in_abweisung_masse

# Funktion def wird ausgeführt
abweisung_in = in_abweisung(data_nw_in)

In [149]:
abweisung_in.describe()

,_key,case_nr,court,date_of_birth,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
count,146,1588,1588,635,1561,1588,1588,1588,1588,1588,1588,1582,0,0,0,0,0
unique,146,798,19,291,130,117,1,818,1,798,1,592,0,0,0,0,0
top,491759/1/11/17266,0085_IN00035_18,agbochum,1971-06-10 00:00:00,2021-06-08 00:00:00,2021-02-17 00:00:00,nw,gerichte/nw/agessen/20/0166_IN00072_20/2021_05...,in,"Ember Watches GmbH, Remscheid, 145 IN 70/20, R...",Abweisungen_mangels_Masse,44866,NaN,NaN,NaN,NaN,NaN
freq,1,4,189,6,30,28,1588,2,1588,4,1588,16,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,1920-09-09 00:00:00,2020-02-20 00:00:00,2021-01-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,2000-02-09 00:00:00,2021-06-18 00:00:00,2021-06-18 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
freq_abweisung_in = abweisung_in.groupby(['date_of_publication'], as_index=False).count() 
freq_abweisung_in.sample(5)

,date_of_publication,_key,case_nr,court,date_of_birth,date_of_proceeding,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
67,2021-04-07,0,14,14,6,14,14,14,14,14,14,14,0,0,0,0,0
7,2021-01-13,22,22,22,10,22,22,22,22,22,22,22,0,0,0,0,0
57,2021-03-23,0,16,16,10,16,16,16,16,16,16,16,0,0,0,0,0
100,2021-05-26,0,10,10,4,10,10,10,10,10,10,10,0,0,0,0,0
74,2021-04-16,0,12,12,8,12,12,12,12,12,12,12,0,0,0,0,0


In [151]:
freq_abweisung_in.to_csv('freq_abweisung_in_publ.csv')

In [152]:
freq_in_nw_abweisung_zipcode = abweisung_in.groupby(['zipcode'], as_index=False).count()
freq_in_nw_abweisung_zipcode.sample(10)

,zipcode,_key,case_nr,court,date_of_birth,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,detail_form_name,format,name,register,request_fingerprint
336,44581,1,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
502,58511,2,4,4,2,4,4,4,4,4,4,4,0,0,0,0,0
389,46244,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
423,48366,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
220,32609,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
199,30488,0,2,2,0,2,2,2,2,2,2,2,0,0,0,0,0
275,41199,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
72,15604,1,2,2,0,2,2,2,2,2,2,2,0,0,0,0,0
218,32591,0,2,2,0,2,2,2,2,2,2,2,0,0,0,0,0
1,10240,0,2,2,0,2,2,2,2,2,2,2,0,0,0,0,0


In [153]:
freq_in_nw_abweisung_zipcode.to_csv('freq_abweisung_in_zipcode.csv')

In [138]:
# freq_in_nw_2['date_of_proceeding'] = pd.to_datetime(freq_in_nw_2['date_of_proceeding'], errors = 'coerce')

In [30]:
data_in_nw.sort_values(by="date_of_publication", ascending=True).head(50)

,_key,case_nr,court,date_of_birth,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint,"(date_of_birth, date_of_proceeding, date_of_publication)"
36169,491759/1/10/36169,0079_IN00010_19,agmuenst,NaT,2020-12-30,2021-01-04,nw,gerichte/nw/agmuenst/19/0079_IN00010_19/2021_0...,in,"Dirk & Sabine Lingenberg OHG, Lüdinghausen, 79...",Bestimmung_Termine,59348,NaN,NaN,NaN,NaN,NaN,NaT
37573,491759/1/10/37573,0252_IN00078_14,agdortmd,1951-03-04,2020-12-29,2021-01-04,nw,gerichte/nw/agdortmd/14/0252_IN00078_14/2021_0...,in,"Moser, Harald, Selm, 252 IN 78/14",Sonstiges,59379,NaN,NaN,NaN,NaN,NaN,1951-03-04
37595,491759/1/10/37595,0073_IN00073_19,agmuenst,NaT,2020-12-29,2021-01-04,nw,gerichte/nw/agmuenst/19/0073_IN00073_19/2021_0...,in,"Musatowa, Nadine, Gronau, 73 IN 73/19",Entscheidungen_im_Verfahren_mit_Termine,48599,NaN,NaN,NaN,NaN,NaN,NaT
37596,491759/1/10/37596,0073_IN00073_19,agmuenst,1984-01-07,2020-12-29,2021-01-04,nw,gerichte/nw/agmuenst/19/0073_IN00073_19/2021_0...,in,"Musatowa, Nadine, Gronau, 73 IN 73/19",Entscheidungen_im_Verfahren_Aufhebung_Einstellung,48599,NaN,NaN,NaN,NaN,NaN,1984-01-07
37597,491759/1/10/37597,0073_IN00073_19,agmuenst,NaT,2021-01-04,2021-01-04,nw,gerichte/nw/agmuenst/19/0073_IN00073_19/2021_0...,in,"Musatowa, Nadine, Gronau, 73 IN 73/19",Entscheidungen_im_Verfahren,48149,NaN,NaN,NaN,NaN,NaN,NaT
37599,491759/1/10/37599,0502_IN00139_18,agddorf,1983-09-30,2020-12-30,2021-01-04,nw,gerichte/nw/agddorf/18/0502_IN00139_18/2021_01...,in,"Mustafic, Robert, Duisburg, 502 IN 139/18",Bestimmung_Termine,47198,NaN,NaN,NaN,NaN,NaN,1983-09-30
37614,491759/1/10/37614,0072_IN00071_14,agmuenst,1954-04-13,2020-12-30,2021-01-04,nw,gerichte/nw/agmuenst/14/0072_IN00071_14/2021_0...,in,"Mühlnikel, Annette, Telgte, 72 IN 71/14",Restschuldbefreiung,48291,NaN,NaN,NaN,NaN,NaN,1954-04-13
37623,491759/1/10/37623,0109_IN00034_16,aghagen,1955-03-31,2020-12-30,2021-01-04,nw,gerichte/nw/aghagen/16/0109_IN00034_16/2021_01...,in,"Müller, Gertrud, Schwelm, 109 IN 34/16",Bestimmung_Termine,58332,NaN,NaN,NaN,NaN,NaN,1955-03-31
37624,491759/1/10/37624,070g_IN00064_20,agkoeln,NaT,2021-01-04,2021-01-04,nw,gerichte/nw/agkoeln/20/070g_IN00064_20/2021_01...,in,"Müller, Jeanette, Köln, 70g IN 64/20",Entscheidungen_im_Verfahren_mit_Termine,51103,NaN,NaN,NaN,NaN,NaN,NaT
37682,491759/1/10/37682,0094_IN00002_14,agkrefd,1967-08-30,2021-01-04,2021-01-04,nw,gerichte/nw/agkrefd/14/0094_IN00002_14/2021_01...,in,"Neweling, Ralf, Duisburg, 94 IN 2/14",Entscheidungen_im_Verfahren_Aufhebung_Einstellung,47055,NaN,NaN,NaN,NaN,NaN,1967-08-30


#### Groupby_zipcode